# Content Based Recommendation

In [5]:
import os
import pandas as pd
import numpy as np

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords


## Data Preprocessing

In [7]:
DATA_DIR = os.path.abspath('./datasets/output/data_2.csv')
df = pd.read_csv(DATA_DIR,index_col=0)
df.head()


,class,text
id,,
0,Interview,"﻿विडम्बना, ओलीको सत्ता सवारी पनि सिंहदरबारमा फ..."
1,Interview,﻿नेपाली कांग्रेसको सभापतिमा रामचन्द्र पौडेलको ...
2,Interview,﻿\nदुर्भाग्य नै भन्नुपर्छ हाम्रो प्राथमिकता पे...
3,Interview,"﻿नयनराज पाण्डे, लेखक\nविमोचन हुनै लागेको तपाईं..."
4,Interview,"﻿- भवन भट्ट, उपाध्यक्ष, गैर आवासीय नेपाली संघ\..."


In [8]:
df.shape

(5475, 2)

In [9]:
df.dropna(inplace=True) # drop rows with invalid values

df['id'] = df.index
df

,class,text,id
id,,,
0,Interview,"﻿विडम्बना, ओलीको सत्ता सवारी पनि सिंहदरबारमा फ...",0
1,Interview,﻿नेपाली कांग्रेसको सभापतिमा रामचन्द्र पौडेलको ...,1
2,Interview,﻿\nदुर्भाग्य नै भन्नुपर्छ हाम्रो प्राथमिकता पे...,2
3,Interview,"﻿नयनराज पाण्डे, लेखक\nविमोचन हुनै लागेको तपाईं...",3
4,Interview,"﻿- भवन भट्ट, उपाध्यक्ष, गैर आवासीय नेपाली संघ\...",4
...,...,...,...
5470,Entertainment,﻿उनले ‘टिमवर्क’ र ‘लिडरसिप’ फिल्ममेकिङको महत्व...,5470
5471,Entertainment,रुपा बनिन् हेरिटेज क्वीन नेपाल\n\n\nराजधानीको ...,5471
5472,Entertainment,﻿प्रकाश र किरणको अर्थ एउटै भए पनि कालो पोथी का...,5472


In [10]:
df.shape

(5475, 3)

## Processing data

In [11]:
nepali_stop_words = set(stopwords.words('nepali'))

# tf = TfidfVectorizer(stop_words=nepali_stop_words,encoding='utf-8',decode_error='ignore')
tf = TfidfVectorizer(tokenizer= lambda x: x.split(" "),
                                  sublinear_tf=True, encoding='utf-8',
                                  decode_error='replace',
                                  max_df=0.5,
                                  min_df=10,
                                  stop_words=nepali_stop_words)

tfidf_matrix = tf.fit_transform(df['text'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

/home/ash/.local/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['कम', 'से'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [12]:
cosine_similarities.shape

(5475, 5475)

In [13]:
for idx, row in df.iterrows():
    try:

        idx = int(idx)
        similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
        similar_items = [(float(cosine_similarities[idx][i]), int(df['id'][i]), str(df['class'][i])) for i in similar_indices]
        results[row['id']] = similar_items[1:]
    except Exception as e: 
        print("ERROR",idx, e)
    
print('Completed!')
similar_indices = cosine_similarities[idx].argsort()[:-100:-1]

Completed!


In [15]:
similar_indices.shape

(99,)

## Displaying Results

In [16]:


def item(id):

    id = str(id)

    return {
        'text': df.loc[int(id)]['text'],
        'class': df.loc[int(id)]['class']
           }

def recommender(item_id, num): 
    print("\n")
    print("**Recommendation System**")
    print("The given news " + "'"+ item(item_id)['text'][:100] + "..." + "'" + " is of Category: " + str(item(item_id)['class'][:100]))
    
    print("\nThe " + str(num) + " most similar news in ascending order is given below: ")
    
    recs = results[int(item_id)][:num]
    for index, rec in enumerate(recs):
        print("\n")
        print(str(index+1) + ") " + "Category: " + str(rec[2]) + "\n\n" + "News: " + "'" + item(rec[1])['text'][:200] + "..." + "'" +" (score:" + str(rec[0]) + ")")

# Extract the news (that the user is currently interacting with) and provide 8 recommended news
recommender(item_id=120, num=10)



**Recommendation System**
The given news '﻿राज्यकोषमा सरकारको स्वेच्छाचारिता रोक्न व्यवस्थापिका संसद्को सार्वजनिक लेखा समिति क्रियाशील भए पनि ...' is of Category: Interview

The 10 most similar news in ascending order is given below: 


1) Category: Interview

News: '﻿
व्यवस्थापिका संसद्को अर्थ समितिका सभापति हुन्, सांसद प्रकाश ज्वाला । नेपाल स्रोत भएर पनि परिचालन गर्न नसक्दा विकासमा पछि परेको उनको बुझाइ छ । अहिले चर्चामा रहेको ‘फास्ट ट्रयाक’ भारतीय कम्पनीलाई दिन ...' (score:0.20815232111931387)


2) Category: Business

News: '﻿
व्यवस्थापिका संसद्को अर्थ समितिका सभापति हुन्, सांसद प्रकाश ज्वाला । नेपाल स्रोत भएर पनि परिचालन गर्न नसक्दा विकासमा पछि परेको उनको बुझाइ छ । अहिले चर्चामा रहेको ‘फास्ट ट्रयाक’ भारतीय कम्पनीलाई दिन ...' (score:0.20815232111931387)


3) Category: Business

News: '﻿
अर्थ मन्त्रालयको नेतृत्व सम्हालिसकेका पाका राजनीतिज्ञ हुन्, राष्ट्रिय प्रजातन्त्र पार्टीका वरिष्ठ नेता डा. प्रकाशचन्द्र लोहनी । अर्थतन्त्रका राम्रा जानकार मानिने लोहनी सरकारमा नभए पनि उनको पार्टी रा

In [17]:
import joblib
joblib.dump(results,'similarity_matrix.joblib')

['similarity_matrix.joblib']